In [1]:
# Copyright(C) 2021 刘珅珅
# Environment: python 3.7
# Date: 2021.10.11
# redis分布式锁

In [16]:
import time
import uuid
from threading import Thread

import redis

In [41]:

def acquire_lock(conn, lock_name, acquire_timeout=3, lock_timeout=2):
    """
    基于 Redis 实现的分布式锁
    
    :param conn: Redis 连接
    :param lock_name: 锁的名称
    :param acquire_timeout: 获取锁的超时时间，默认 3 秒
    :param lock_timeout: 锁的超时时间，默认 2 秒
    """
    # 这个采用uuid生成的标识符，主要是在release_lock时，不要删除掉不是由自己创建的锁
    identifier = str(uuid.uuid4())
    end = time.time() + acquire_timeout
    lock_name = "string:lock:{}".format(lock_name)
    while time.time() < end:
        """
        获取锁如果没有超时，其它客户端释放锁时，会删除掉名称为lock_name的key，客户端就能重新设置
        一个名称为lock_name的key
        """
        if conn.set(lock_name, identifier, ex=lock_timeout, nx=True):
            return identifier
        time.sleep(0.001)
    return ""



In [42]:
def release_lock(conn, lock_name, identifier):
    """
    释放锁
    
    :param conn: Redis 连接
    :param lockname: 锁的名称
    :param identifier: 锁的标识
    :return:
    """
    unlock_script = """
    if redis.call("get",KEYS[1]) == ARGV[1] then
        return redis.call("del",KEYS[1])
    else
        return 0
    end
    """
    lock_name = "string:lock:{}".format(lock_name)
    # 调用redis内嵌的lua脚本，保证get和delete的原子操作
    unlock = conn.register_script(unlock_script)
    result = unlock(keys=[lock_name], args=[identifier])
    if result:
        return True
    else:
        return False

In [43]:
conn = redis.Redis(host='localhost', port=6379, db=0)

In [49]:
count = 10

def run(i):
    identifier = acquire_lock(conn, 'resource')
    if len(identifier) == 0:
        return
    
    print("线程:{}--获得了锁\n".format(i))
    time.sleep(0.01)
    global count
    if count < 1:
        print("线程:{}--没抢到，票抢完了\n".format(i))
        return
    count -= 1
    print("线程:{}--抢到一张票，还剩{}张票\n".format(i, count))
    release_lock(conn, 'resource', identifier)
            

In [50]:
for i in range(50):
    t = Thread(target=run,args=(i,))
    t.start()

线程:0--获得了锁

线程:0--抢到一张票，还剩9张票

线程:6--获得了锁

线程:6--抢到一张票，还剩8张票

线程:7--获得了锁

线程:7--抢到一张票，还剩7张票

线程:17--获得了锁

线程:17--抢到一张票，还剩6张票

线程:3--获得了锁

线程:3--抢到一张票，还剩5张票

线程:16--获得了锁

线程:16--抢到一张票，还剩4张票

线程:11--获得了锁

线程:11--抢到一张票，还剩3张票

线程:21--获得了锁

线程:21--抢到一张票，还剩2张票

线程:43--获得了锁

线程:43--抢到一张票，还剩1张票

线程:48--获得了锁

线程:48--抢到一张票，还剩0张票

线程:32--获得了锁

线程:32--没抢到，票抢完了

线程:34--获得了锁

线程:34--没抢到，票抢完了

